# Real-time Steps

In real-time mode, the FPGA lets Xylo run freely and only performs readout after receiving a timestep done interrupt.
This means sending input to Xylo from the FPGA is not supported in real-time mode.  Instead, the switches on the PCB must be configured such that Xylo receives input from the microphones on the PCB.


## Simple Samna example to access the board

In [22]:
ts_duration   = 1000*1e-3; # in second
main_clk_freq = 50*1e6 # in Hz
tr_wrap       = int(ts_duration*main_clk_freq)
print(tr_wrap)

50000000


In [23]:
import samna
print(f"samna version: {samna.__version__}")
import time

device = samna.device.open_device('XyloAudio3TestBoard')

# set device configuration
ioc = device.get_io_control_module()
io = device.get_io_module()
model = device.get_model()

# power off
ioc.write_config(0, 0x0)
time.sleep(1)

# main clock disable
io.write_config(0x0008, 0)
time.sleep(1)

# power on
ioc.write_config(0, 0x1B)
time.sleep(1)

# main clock enable
io.write_config(0x0008, 1)
time.sleep(1)

# DIG_MIC drives PDM_DATA
io.write_config(0x0012, 0)
time.sleep(1)

# define real-time mode
io.write_config(0x31, 2)
time.sleep(1)

# set samna configuration
config = samna.xyloAudio3.configuration.XyloConfiguration()

config.operation_mode = samna.xyloAudio3.OperationMode.RealTime
config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm
config.time_resolution_wrap = 50000000
config.debug.debug_status_update_enable = 1
config.debug.debug_input_enable = 1
config.debug.monitor_signals_enable = 1
# 0: input for Xylo, 1: Xylo output
config.digital_frontend.pdm_preprocessing.clock_direction = 1
# 0: Xylo samples PDM_DATA at risinge edge, 1: Xylo samples PDM_DATA at falling edge
config.digital_frontend.pdm_preprocessing.clock_edge = 1

device.get_model().apply_configuration(config)

# create access to xylo input/output
source = samna.graph.source_to(model.get_sink_node())
sink   = samna.graph.sink_from(model.get_source_node())

source.write([samna.xyloAudio3.event.ReadRegisterValue(12)])
print(sink.get_n_events(1, 2000))


samna version: 0.37.10.26+g11d2fc36e.dirty


RuntimeError: No devices available!

## Example with Rockpool

Open Dev Kit

* Call open_device() on a discovered device with Samna.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import rockpool
print(f'Rockpool version {rockpool.__version__}')
import samna
print(f'Samna version {samna.__version__}')
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper, XyloMonitor, XyloSamna

from rockpool.devices.xylo import find_xylo_hdks

from rockpool.transform.quantize_methods import channel_quantize
from rockpool.nn.modules import LIF, Linear
from rockpool.nn.combinators import Sequential

import numpy as np

# find_xylo_hdks() directly calls the Samna device finder and opens all available devices.
# When Samna opens a device, it will perform actions on the board. Such as, powering on the chip, resetting it,
# applying a default configuration, and configuring the FPGA for default operation.

xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()
print(xylo_hdks, xylo_support_modules, xylo_versions)

Rockpool version 2.7.dev
Samna version 0.37.10.26+g11d2fc36e.dirty
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a Xylo A3. Importing `rockpool.devices.xylo.syns65302`
[<samna.xyloAudio3Boards.XyloAudio3TestBoard object at 0x7546c989baf0>] [<module 'rockpool.devices.xylo.syns65302' from '/home/vleite/Software/rockpool/rockpool/devices/xylo/syns65302/__init__.py'>] ['syns65302']


## Build a model and map it to hardware

In [2]:
# simple model to test

net = Sequential(
    Linear((16, 63)),
    LIF((63, 63)),

    Linear((63, 32)),
    LIF(32),
)

print(net)

# net[0].weight *= 0.05
spec = mapper(net.as_graph())
Q_spec = spec
Q_spec.update(channel_quantize(**Q_spec))
config, is_valid, msg = config_from_specification(**Q_spec)

if not is_valid:
    print(msg)

ModSequential  with shape (16, 32) {
    Linear '0_Linear' with shape (16, 63)
    LIF '1_LIF' with shape (63, 63)
    Linear '2_Linear' with shape (63, 32)
    LIF '3_LIF' with shape (32, 32)
}


## Set up Samna graphs and configuration

* Create a buffer sink for events from the chip model source node. Readout events will be captured here.
* Set Xylo configuration as desired for PDM and network config
* Set Xylo TR_WRAP register for desired time step length with default 25 MHz clock
* Set Xylo to real-time mode
* Enable PCB switches for PDM microphones
* Set FPGA module to real-time mode
   * io.write_config(0x12, 0) and io.write_config(0x31, 2)

In [3]:
# create xylo monitor
dt = 0.01
xylo_node = xylo_hdks[0]
print(xylo_node)


# PDM
config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm

# TR_WRAP register
# config.time_resolution_wrap = int(0x7_9FF3)
config.time_resolution_wrap = 50000000 # use the same value as in Xin's tests

# real-time mode
config.operation_mode = samna.xyloAudio3.OperationMode.RealTime

config.debug.debug_status_update_enable = 1
config.debug.debug_input_enable = 1
config.debug.monitor_signals_enable = 1

# PCB switches
# This is physically checked on the board

# inside XyloMonitor the buffers are created and FPGA modules are set
xylo_monitor = XyloMonitor(device=xylo_node, 
    config=config, 
    dt = dt,
    output_mode='Spike')

# print(xylo_monitor)

print(xylo_monitor.config)

# Create a XyloSamna structure for evolving
modSamna = XyloSamna(xylo_node, config, dt = dt)

xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(0)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))


xyloAudio3::configuration::XyloConfiguration(operation_mode=OperationMode.RealTime, bias_enable=1, time_resolution_wrap=499699, input=xyloAudio3::configuration::InputConfig(weight_bit_shift=0, weights={ -119 -13 79 -97 -99 44 124 -2 -61 23 88 -32 -119 30 76 -3 127 -107 -47 127 40 -34 111 -104 -126 102 -10 121 -75 4 -116 -127 -127 12 -1 93 -117 -90 -38 104 127 -107 -17 21 -75 -40 -63 24 -127 57 61 -53 41 105 -127 -117 -9 65 27 -67 94 83 83 65 112 109 -77 -47 -60 57 121 -66 100 40 84 -127 -18 23 60 59 64 -37 114 -13 85 -14 -118 -53 97 98 -83 -27 56 32 -41 20 107 71 78 93 -127 15 14 -41 -57 -121 -58 119 -3 -9 56 -38 -71 -116 -127 46 -113 -117 -40 68 -113 33 -87 55 85 122 -51 -36 -71 -19 16 45 -101 -117 -107 97 -97 127 53 97 48 -81 105 -123 -119 103 90 -62 -97 113 -116 103 80 -19 -7 -78 127 -16 109 -38 55 -71 -51 42 -13 127 -15 112 -34 -80 40 51 -124 116 111 86 -72 25 59 100 98 -78 -86 42 102 -127 63 -41 49 107 66 -113 49 13 18 86 84 21 -17 -43 60 -88 29 39 -17 40 -101 73 -7 -15 -18 30 -88

## Start processing and collect events

* Send TriggerProcessing event from Samna with the desired time step
* Collect readout events in the BufferSink attached to the model source node

In [4]:
# The notebook calls XyloMonitor.evolve() with some input data, which the evolve function will try to send. 
# This is not allowed in real-time mode.

import numpy as np
N = 1000

# in rd there is a AFE related record ('spike_in' or similar).
# It should be empty in silence and non empty when we make noise
out, _, rd = modSamna.evolve(input = np.zeros((N,2)), record = True)
print(rd)

# out, _, rec = xylo_monitor.evolve(input_data=np.ones((N,16)))
print(out)


  0%|          | 0/1000 [00:00<?, ?it/s]

{'Vmem': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16), 'Isyn': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16), 'Isyn2': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16), 'Spikes': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, 

In [21]:
hex(1303)


'0x517'

In [23]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(0)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[xyloAudio3::event::MemoryValue(address=4623, data=0)]


In [18]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(12)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[xyloAudio3::event::MemoryValue(address=4619, data=0)]


In [19]:
xylo_monitor._write_buffer.write([samna.xyloAudio3.event.ReadRegisterValue(27)])
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[xyloAudio3::event::MemoryValue(address=4620, data=0)]


In [20]:
xylo_monitor._io.write_config(0x0008, 1)
print(xylo_monitor._read_buffer.get_n_events(1, 2000))

[xyloAudio3::event::MemoryValue(address=4621, data=0)]
